# **Method 3: Dynamic Client Selection with Accuracy Threshold**

This method introduces an accuracy threshold to exclude high-performing clients (accuracy ≥ 0.9) from subsequent rounds, focusing resources on weaker-performing clients.

In [ ]:
!pip install flwr
!pip install torch
!pip install matplotlib
!pip install torchvision
!pip install flwr-datasets[vision]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 523.6/523.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.68.1
    Uninstalling grpcio-1.68.1:
      Successfully uninstalled grpcio-1.68.1
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall: typer
    Found existing installation: typer 0.15.1
    Uninstalling typer-0.15.1:
      Successfully uninstalled typer-0.15.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
!pip install -q flwr[simulation] flwr-datasets[vision] torch torchvision matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 11.0 MB/s eta 0:00:00


In [ ]:
# Import Libs
from collections import OrderedDict
from typing import List, Tuple, Dict, Optional

import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader

import flwr as fl
from flwr.client import Client, NumPyClient
from flwr.common import Metrics, Parameters, Scalar, FitIns, FitRes, EvaluateRes, Config
from flwr.server import ServerConfig
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy import FedAvg
from flwr_datasets import FederatedDataset

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Device configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

# Disable progress bar from datasets
disable_progress_bar()


Training on cpu


In [ ]:
# Global Var
NUM_CLIENTS = 10
NUM_ROUNDS = 20
BATCH_SIZE = 32

In [ ]:
# Function to load MNIST Dataset
def load_datasets(partition_id: int):
    fds = FederatedDataset(dataset="mnist", partitioners={"train": NUM_CLIENTS})
    partition = fds.load_partition(partition_id)
    # Divide data on each node: 80% train, 20% test
    partition_train_test = partition.train_test_split(test_size=0.2, seed=42)
    pytorch_transforms = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))] # Transformation for MNIST
    )

    def apply_transforms(batch):
        batch["image"] = [pytorch_transforms(img) for img in batch["image"]]
        return batch

    # Create training and validation for each partition and wrap it into DataLoader
    partition_train_test = partition_train_test.with_transform(apply_transforms)
    trainloader = DataLoader(
        partition_train_test["train"], batch_size=BATCH_SIZE, shuffle=True
    )
    valloader = DataLoader(partition_train_test["test"], batch_size=BATCH_SIZE)
    testset = fds.load_split("test").with_transform(apply_transforms)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloader, valloader, testloader

In [ ]:
# CNN Model for MNIST
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)   # 1 input channel (grayscale)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 16 channels * 4 * 4 = 256
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)  # Flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
# Training function
def train(net, trainloader, epochs: int):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for _ in range(epochs):
        for batch in trainloader:
            images, labels = batch["image"].to(DEVICE), batch["label"].to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# Testing function
def test(net, testloader):
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch["image"].to(DEVICE), batch["label"].to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy


In [ ]:
# Functions to set and get model parameters
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

In [ ]:
# Flower client implementation
class FlowerClient(NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config: Config) -> List[np.ndarray]:
        return get_parameters(self.net)

    def fit(self, parameters: List[np.ndarray], config: Config) -> Tuple[List[np.ndarray], int, Dict]:
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader.dataset), {}

    def evaluate(self, parameters: List[np.ndarray], config: Config) -> Tuple[float, int, Dict]:
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader.dataset), {"accuracy": float(accuracy)}

In [ ]:
# Client function
def client_fn(cid: str) -> Client:


    net = Net().to(DEVICE)
    # Load data (each client has its own partition)
    partition_id = int(cid)
    trainloader, valloader, _ = load_datasets(partition_id)
    # Create and return the Flower client
    return FlowerClient(net, trainloader, valloader)

Strategy for Method 3

In [ ]:
# Client selection with accuracy threshold, excludes clients above threshold and those with accuracy = 1.0
class CustomFedAvg(FedAvg):
    def __init__(self, accuracy_threshold=0.9, **kwargs):
        super().__init__(**kwargs)
        self.client_accuracies: Dict[str, float] = {}
        self.client_probabilities: Dict[str, float] = {}
        self.metrics_distributed = {"accuracy": []}
        self.latest_parameters: Optional[Parameters] = None
        self.communication_count = 0
        self.terminated = False
        self.accuracy_threshold = accuracy_threshold

    def initialize_clients(self, client_manager: ClientManager):
        for client_proxy in client_manager.all().values():
            cid = client_proxy.cid
            self.client_accuracies[cid] = 0.0
            self.client_probabilities[cid] = 1.0

    def configure_fit(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager,
    ) -> List[Tuple[ClientProxy, FitIns]]:
        if self.terminated:
            return []

        if server_round == 1:
            self.initialize_clients(client_manager)

        all_clients = list(client_manager.all().values())
        selected_clients = []

        print(f"\nRound {server_round}:")
        for client in all_clients:
            cid = client.cid

            current_accuracy = self.client_accuracies.get(cid, 0.0)

            if current_accuracy >= self.accuracy_threshold:
                self.client_probabilities[cid] = 0.0
                print(f"  Client {cid}: accuracy {current_accuracy:.4f} >= threshold {self.accuracy_threshold}, excluded")
                continue

            # Calculate selection probability p_i(t+1) = 1 - a_i(t)
            probability = 1.0 - current_accuracy
            probability = max(0.0, probability)  # Make sure probability is not negative
            self.client_probabilities[cid] = probability

            x = random.uniform(0, 1)
            is_selected = x <= probability
            selection_status = "selected" if is_selected else "not selected"
            print(f"  Client {cid}: p_i({server_round}) = {probability:.4f}, x = {x:.4f}, {selection_status}")
            if is_selected:
                selected_clients.append(client)

        if not selected_clients:
            print("All clients have reached the accuracy threshold. Terminating training.")
            self.terminated = True
            return []

        self.communication_count += len(selected_clients)
        print(f"  Number of clients selected: {len(selected_clients)}")

        config = {}
        fit_ins = FitIns(parameters, config)
        return [(client, fit_ins) for client in selected_clients]

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(server_round, results, failures)
        if aggregated_parameters is not None:
            self.latest_parameters = aggregated_parameters
        return aggregated_parameters, aggregated_metrics

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[BaseException],
    ) -> Optional[float]:
        metrics = []
        participating_cids = [client_proxy.cid for client_proxy, _ in results]

        for client_proxy, evaluate_res in results:
            cid = client_proxy.cid
            num_examples = evaluate_res.num_examples
            accuracy = evaluate_res.metrics.get("accuracy", 0.0)
            self.client_accuracies[cid] = accuracy
            metrics.append((num_examples, {"accuracy": accuracy}))

        print(f"\nRound {server_round} client accuracies:")
        for cid in self.client_accuracies.keys():
            accuracy = self.client_accuracies[cid]
            status = "participated" if cid in participating_cids else "did not participate"
            print(f"  Client {cid}: accuracy = {accuracy:.4f} ({status})")

        aggregated_metrics = self.weighted_average(metrics)
        global_accuracy = aggregated_metrics["accuracy"]
        self.metrics_distributed["accuracy"].append((server_round, global_accuracy))
        print(f"Round {server_round} aggregated accuracy: {global_accuracy:.4f}")
        return super().aggregate_evaluate(server_round, results, failures)

    @staticmethod
    def weighted_average(metrics: List[Tuple[int, Dict[str, Scalar]]]) -> Dict[str, Scalar]:
        accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
        examples = [num_examples for num_examples, _ in metrics]
        if sum(examples) == 0:
            return {"accuracy": 0.0}
        return {"accuracy": sum(accuracies) / sum(examples)}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
NUM_CLIENTS = 10
NUM_ROUNDS = 20
ACCURACY_THRESHOLD = 0.9  # adjust this threshold as needed

strategy1 = CustomFedAvg(
    accuracy_threshold=ACCURACY_THRESHOLD,
    fraction_fit=1.0,         # All clients are considered in selection
    min_fit_clients=1,        # At least one client should be available
    min_available_clients=NUM_CLIENTS,
)


In [ ]:
server_config1 = ServerConfig(num_rounds=NUM_ROUNDS)

In [ ]:
# Start the simulation
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    client_resources={"num_gpus": 0.5 if torch.cuda.is_available() else 0},
    config=server_config1,
    strategy=strategy1,
)

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout
2024-12-27 09:26:17,979	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'memory': 7962127566.0, 'object_store_memory': 3981063782.0, 'node:__internal_head__': 1.0, 'GPU': 1.0, 'node:172.28.0.12': 1.0, 'accelerator_type:T4': 1.0, 'CPU': 2.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
<ipython-input-18-07c849282ceb>:2: UserWarning: No `num_cpus` specified in `client_resources`. Using `num_c


Round 1:
  Client 12138372675816268685: p_i(1) = 1.0000, x = 0.2656, selected
  Client 9494354600812815423: p_i(1) = 1.0000, x = 0.1309, selected
  Client 202973983290458397: p_i(1) = 1.0000, x = 0.6455, selected
  Client 17841402951903752161: p_i(1) = 1.0000, x = 0.4572, selected
  Client 6184363314765092041: p_i(1) = 1.0000, x = 0.9290, selected
  Client 11481533082800000438: p_i(1) = 1.0000, x = 0.9357, selected
  Client 17018922978604655791: p_i(1) = 1.0000, x = 0.0093, selected
  Client 1022704300547147465: p_i(1) = 1.0000, x = 0.6212, selected
  Client 6586524741957697603: p_i(1) = 1.0000, x = 0.5630, selected
  Client 8326844063857993821: p_i(1) = 1.0000, x = 0.1000, selected
  Number of clients selected: 10


(ClientAppActor pid=7534) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(ClientAppActor pid=7534) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=7534) 
(ClientAppActor pid=7534)             This is a deprecated feature. It will be removed
(ClientAppActor pid=7534)             entirely in future versions of Flower.
(ClientAppActor pid=7534)         
(ClientAppActor pid=7533) /usr/local/lib/python3.10/dist-packages/jupyter_client/connect.py:28: DeprecationWarning: Jupyter is migrating its paths to use standard platformdirs
(ClientAppActor pid=7533) given by the platformdirs library.  To remove this warning


Round 1 client accuracies:
  Client 12138372675816268685: accuracy = 0.9150 (participated)
  Client 9494354600812815423: accuracy = 0.9233 (participated)
  Client 202973983290458397: accuracy = 0.9283 (participated)
  Client 17841402951903752161: accuracy = 0.9192 (participated)
  Client 6184363314765092041: accuracy = 0.9150 (participated)
  Client 11481533082800000438: accuracy = 0.9200 (participated)
  Client 17018922978604655791: accuracy = 0.9117 (participated)
  Client 1022704300547147465: accuracy = 0.9208 (participated)
  Client 6586524741957697603: accuracy = 0.9133 (participated)
  Client 8326844063857993821: accuracy = 0.9300 (participated)
Round 1 aggregated accuracy: 0.9197

Round 2:
  Client 12138372675816268685: accuracy 0.9150 >= threshold 0.9, excluded
  Client 9494354600812815423: accuracy 0.9233 >= threshold 0.9, excluded
  Client 202973983290458397: accuracy 0.9283 >= threshold 0.9, excluded
  Client 17841402951903752161: accuracy 0.9192 >= threshold 0.9, excluded


(ClientAppActor pid=7533) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 5x across cluster]
(ClientAppActor pid=7533)          [repeated 10x across cluster]
(ClientAppActor pid=7533)             This is a deprecated feature. It will be removed [repeated 5x across cluster]
(ClientAppActor pid=7533)             entirely in future versions of Flower. [repeated 5x across cluster]
(ClientAppActor pid=7533) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 5x across cluster]
(ClientAppActor pid=7533) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided 

...


Round 18 client accuracies:
  Client 12138372675816268685: accuracy = 0.9150 (participated)
  Client 9494354600812815423: accuracy = 0.9233 (participated)
  Client 202973983290458397: accuracy = 0.9283 (participated)
  Client 17841402951903752161: accuracy = 0.9192 (participated)
  Client 6184363314765092041: accuracy = 0.9150 (participated)
  Client 11481533082800000438: accuracy = 0.9200 (participated)
  Client 17018922978604655791: accuracy = 0.9117 (participated)
  Client 1022704300547147465: accuracy = 0.9208 (participated)
  Client 6586524741957697603: accuracy = 0.9133 (participated)
  Client 8326844063857993821: accuracy = 0.9300 (participated)
Round 18 aggregated accuracy: 0.9197


(ClientAppActor pid=7533) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 4x across cluster]
(ClientAppActor pid=7533)          [repeated 8x across cluster]
(ClientAppActor pid=7533)             This is a deprecated feature. It will be removed [repeated 4x across cluster]
(ClientAppActor pid=7533)             entirely in future versions of Flower. [repeated 4x across cluster]
(ClientAppActor pid=7533) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 5x across cluster]
(ClientAppActor pid=7534) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `


Round 19 client accuracies:
  Client 12138372675816268685: accuracy = 0.9150 (participated)
  Client 9494354600812815423: accuracy = 0.9233 (participated)
  Client 202973983290458397: accuracy = 0.9283 (participated)
  Client 17841402951903752161: accuracy = 0.9192 (participated)
  Client 6184363314765092041: accuracy = 0.9150 (participated)
  Client 11481533082800000438: accuracy = 0.9200 (participated)
  Client 17018922978604655791: accuracy = 0.9117 (participated)
  Client 1022704300547147465: accuracy = 0.9208 (participated)
  Client 6586524741957697603: accuracy = 0.9133 (participated)
  Client 8326844063857993821: accuracy = 0.9300 (participated)
Round 19 aggregated accuracy: 0.9197


(ClientAppActor pid=7534) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 4x across cluster]
(ClientAppActor pid=7534)          [repeated 8x across cluster]
(ClientAppActor pid=7534)             This is a deprecated feature. It will be removed [repeated 4x across cluster]
(ClientAppActor pid=7534)             entirely in future versions of Flower. [repeated 4x across cluster]
(ClientAppActor pid=7534) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 4x across cluster]
(ClientAppActor pid=7533) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `


Round 20 client accuracies:
  Client 12138372675816268685: accuracy = 0.9150 (participated)
  Client 9494354600812815423: accuracy = 0.9233 (participated)
  Client 202973983290458397: accuracy = 0.9283 (participated)
  Client 17841402951903752161: accuracy = 0.9192 (participated)
  Client 6184363314765092041: accuracy = 0.9150 (participated)
  Client 11481533082800000438: accuracy = 0.9200 (participated)
  Client 17018922978604655791: accuracy = 0.9117 (participated)
  Client 1022704300547147465: accuracy = 0.9208 (participated)
  Client 6586524741957697603: accuracy = 0.9133 (participated)
  Client 8326844063857993821: accuracy = 0.9300 (participated)
Round 20 aggregated accuracy: 0.9197


In [ ]:
# Total Communication round
print(f"\nTotal communication counts (number of client participations): {strategy1.communication_count}")



Total communication counts (number of client participations): 10


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# **Method 4: Client Selection Based on Performance Comparison**

This method selects clients whose accuracy has not improved over previous rounds, prioritizing underperforming clients to enhance global model convergence.

In [ ]:
# Selects underperforming clients (current accuracy <= previous accuracy), terminates when all clients are performing well.
from typing import Dict, List, Optional, Tuple

import flwr as fl
from flwr.common import (
    Parameters,
    Scalar,
    FitIns,
    FitRes,
    EvaluateRes,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy

class Aquib(fl.server.strategy.FedAvg):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.client_accuracies: Dict[str, float] = {}
        self.previous_client_accuracies: Dict[str, Optional[float]] = {}
        self.communication_count = 0
        self.terminated = False

    def initialize_clients(self, client_manager: ClientManager):
        for client_proxy in client_manager.all().values():
            cid = client_proxy.cid
            # Initialize previous accuracies to None
            self.previous_client_accuracies[cid] = None
            self.client_accuracies[cid] = 0.0

    def configure_fit(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager,
    ) -> List[Tuple[ClientProxy, FitIns]]:
        if self.terminated:
            return []

        if server_round == 1:
            self.initialize_clients(client_manager)

        all_clients = list(client_manager.all().values())
        selected_clients = []

        print(f"\nRound {server_round}:")
        for client in all_clients:
            cid = client.cid

            current_accuracy = self.client_accuracies.get(cid, 0.0)
            previous_accuracy = self.previous_client_accuracies.get(cid)

            # Determine if the client is underperforming
            if previous_accuracy is None:
                # First round, consider all clients as underperforming
                underperforming = True
            else:
                if current_accuracy <= previous_accuracy:
                    underperforming = True
                else:
                    underperforming = False

            if underperforming:
                # Client is underperforming, select for training
                print(f"  Client {cid}: Underperforming (prev: {previous_accuracy}, current: {current_accuracy}), selected")
                selected_clients.append(client)
            else:
                # Client is performing well, do not select for training
                print(f"  Client {cid}: Performing well (prev: {previous_accuracy}, current: {current_accuracy}), not selected")

        if not selected_clients:
            print("All clients are performing well. Terminating training.")
            self.terminated = True
            return []

        self.communication_count += len(selected_clients)
        print(f"  Number of clients selected: {len(selected_clients)}")

        # Prepare fit instructions
        config = {}
        fit_ins = FitIns(parameters, config)
        return [(client, fit_ins) for client in selected_clients]

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        # Call super to aggregate parameters as usual
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(server_round, results, failures)
        if aggregated_parameters is not None:
            self.latest_parameters = aggregated_parameters
        return aggregated_parameters, aggregated_metrics

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[BaseException],
    ) -> Optional[float]:
        metrics = []
        participating_cids = [client_proxy.cid for client_proxy, _ in results]

        for client_proxy, evaluate_res in results:
            cid = client_proxy.cid
            num_examples = evaluate_res.num_examples
            accuracy = evaluate_res.metrics.get("accuracy", 0.0)

            # Update previous accuracy
            self.previous_client_accuracies[cid] = self.client_accuracies.get(cid, 0.0)
            # Update current accuracy
            self.client_accuracies[cid] = accuracy

            metrics.append((num_examples, {"accuracy": accuracy}))

        print(f"\nRound {server_round} client accuracies:")
        for cid in self.client_accuracies.keys():
            current_accuracy = self.client_accuracies[cid]
            previous_accuracy = self.previous_client_accuracies[cid]
            status = "participated" if cid in participating_cids else "did not participate"
            print(f"  Client {cid}: accuracy = {current_accuracy:.4f} (prev: {previous_accuracy}, {status})")

        aggregated_metrics = self.weighted_average(metrics)
        global_accuracy = aggregated_metrics["accuracy"]
        print(f"Round {server_round} aggregated accuracy: {global_accuracy:.4f}")

        return super().aggregate_evaluate(server_round, results, failures)

    @staticmethod
    def weighted_average(metrics: List[Tuple[int, Dict[str, Scalar]]]) -> Dict[str, Scalar]:
        accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
        examples = [num_examples for num_examples, _ in metrics]
        if sum(examples) == 0:
            return {"accuracy": 0.0}
        return {"accuracy": sum(accuracies) / sum(examples)}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
NUM_CLIENTS = 10
NUM_ROUNDS = 20

strategy4 = Aquib(
    fraction_fit=1.0,         # All clients are considered in selection
    min_fit_clients=1,        # At least one client should be available
    min_available_clients=NUM_CLIENTS,
)


In [ ]:
server_config4 = ServerConfig(num_rounds=NUM_ROUNDS)

# Start the simulation
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    client_resources={"num_gpus": 0.5 if torch.cuda.is_available() else 0},
    config=server_config4,
    strategy=strategy4,
)

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout
2024-12-27 09:40:42,819	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'object_store_memory': 3982357708.0, 'memory': 7964715419.0, 'CPU': 2.0, 'accelerator_type:T4': 1.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
<ipython-input-28-93801a22dc50>:4: UserWarning: No `num_cpus` specified in `client_resources`. Using `num_c


Round 1:
  Client 9548164061767271908: Underperforming (prev: None, current: 0.0), selected
  Client 8492160752674190605: Underperforming (prev: None, current: 0.0), selected
  Client 142559942568206738: Underperforming (prev: None, current: 0.0), selected
  Client 2091763657617065565: Underperforming (prev: None, current: 0.0), selected
  Client 18318300298840459521: Underperforming (prev: None, current: 0.0), selected
  Client 12729532527466555916: Underperforming (prev: None, current: 0.0), selected
  Client 1791549574661214653: Underperforming (prev: None, current: 0.0), selected
  Client 17788363131690628232: Underperforming (prev: None, current: 0.0), selected
  Client 5822289512950929004: Underperforming (prev: None, current: 0.0), selected
  Client 13263297433801258679: Underperforming (prev: None, current: 0.0), selected
  Number of clients selected: 10


(ClientAppActor pid=17363) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(ClientAppActor pid=17363) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=17363) 
(ClientAppActor pid=17363)             This is a deprecated feature. It will be removed
(ClientAppActor pid=17363)             entirely in future versions of Flower.
(ClientAppActor pid=17363)         
(ClientAppActor pid=17362) /usr/local/lib/python3.10/dist-packages/jupyter_client/connect.py:28: DeprecationWarning: Jupyter is migrating its paths to use standard platformdirs
(ClientAppActor pid=17362) given by the platformdirs library.  To remove this


Round 1 client accuracies:
  Client 9548164061767271908: accuracy = 0.9025 (prev: 0.0, participated)
  Client 8492160752674190605: accuracy = 0.9008 (prev: 0.0, participated)
  Client 142559942568206738: accuracy = 0.9033 (prev: 0.0, participated)
  Client 2091763657617065565: accuracy = 0.8933 (prev: 0.0, participated)
  Client 18318300298840459521: accuracy = 0.8975 (prev: 0.0, participated)
  Client 12729532527466555916: accuracy = 0.8975 (prev: 0.0, participated)
  Client 1791549574661214653: accuracy = 0.8950 (prev: 0.0, participated)
  Client 17788363131690628232: accuracy = 0.8975 (prev: 0.0, participated)
  Client 5822289512950929004: accuracy = 0.8992 (prev: 0.0, participated)
  Client 13263297433801258679: accuracy = 0.9217 (prev: 0.0, participated)
Round 1 aggregated accuracy: 0.9008

Round 2:
  Client 9548164061767271908: Performing well (prev: 0.0, current: 0.9025), not selected
  Client 8492160752674190605: Performing well (prev: 0.0, current: 0.9008333333333334), not se

(ClientAppActor pid=17362) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 4x across cluster]
(ClientAppActor pid=17362)          [repeated 8x across cluster]
(ClientAppActor pid=17362)             This is a deprecated feature. It will be removed [repeated 4x across cluster]
(ClientAppActor pid=17362)             entirely in future versions of Flower. [repeated 4x across cluster]
(ClientAppActor pid=17362) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 4x across cluster]
(ClientAppActor pid=17362) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The prov

...


Round 18 client accuracies:
  Client 9548164061767271908: accuracy = 0.9025 (prev: 0.9025, participated)
  Client 8492160752674190605: accuracy = 0.9008 (prev: 0.9008333333333334, participated)
  Client 142559942568206738: accuracy = 0.9033 (prev: 0.9033333333333333, participated)
  Client 2091763657617065565: accuracy = 0.8933 (prev: 0.8933333333333333, participated)
  Client 18318300298840459521: accuracy = 0.8975 (prev: 0.8975, participated)
  Client 12729532527466555916: accuracy = 0.8975 (prev: 0.8975, participated)
  Client 1791549574661214653: accuracy = 0.8950 (prev: 0.895, participated)
  Client 17788363131690628232: accuracy = 0.8975 (prev: 0.8975, participated)
  Client 5822289512950929004: accuracy = 0.8992 (prev: 0.8991666666666667, participated)
  Client 13263297433801258679: accuracy = 0.9217 (prev: 0.9216666666666666, participated)
Round 18 aggregated accuracy: 0.9008


(ClientAppActor pid=17362) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 4x across cluster]
(ClientAppActor pid=17362)          [repeated 8x across cluster]
(ClientAppActor pid=17362)             This is a deprecated feature. It will be removed [repeated 4x across cluster]
(ClientAppActor pid=17362)             entirely in future versions of Flower. [repeated 4x across cluster]
(ClientAppActor pid=17362) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 4x across cluster]
(ClientAppActor pid=17362) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The prov


Round 19 client accuracies:
  Client 9548164061767271908: accuracy = 0.9025 (prev: 0.9025, participated)
  Client 8492160752674190605: accuracy = 0.9008 (prev: 0.9008333333333334, participated)
  Client 142559942568206738: accuracy = 0.9033 (prev: 0.9033333333333333, participated)
  Client 2091763657617065565: accuracy = 0.8933 (prev: 0.8933333333333333, participated)
  Client 18318300298840459521: accuracy = 0.8975 (prev: 0.8975, participated)
  Client 12729532527466555916: accuracy = 0.8975 (prev: 0.8975, participated)
  Client 1791549574661214653: accuracy = 0.8950 (prev: 0.895, participated)
  Client 17788363131690628232: accuracy = 0.8975 (prev: 0.8975, participated)
  Client 5822289512950929004: accuracy = 0.8992 (prev: 0.8991666666666667, participated)
  Client 13263297433801258679: accuracy = 0.9217 (prev: 0.9216666666666666, participated)
Round 19 aggregated accuracy: 0.9008


(ClientAppActor pid=17363) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 5x across cluster]
(ClientAppActor pid=17363)          [repeated 10x across cluster]
(ClientAppActor pid=17363)             This is a deprecated feature. It will be removed [repeated 5x across cluster]
(ClientAppActor pid=17363)             entirely in future versions of Flower. [repeated 5x across cluster]
(ClientAppActor pid=17362) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 5x across cluster]
(ClientAppActor pid=17362) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The pro


Round 20 client accuracies:
  Client 9548164061767271908: accuracy = 0.9025 (prev: 0.9025, participated)
  Client 8492160752674190605: accuracy = 0.9008 (prev: 0.9008333333333334, participated)
  Client 142559942568206738: accuracy = 0.9033 (prev: 0.9033333333333333, participated)
  Client 2091763657617065565: accuracy = 0.8933 (prev: 0.8933333333333333, participated)
  Client 18318300298840459521: accuracy = 0.8975 (prev: 0.8975, participated)
  Client 12729532527466555916: accuracy = 0.8975 (prev: 0.8975, participated)
  Client 1791549574661214653: accuracy = 0.8950 (prev: 0.895, participated)
  Client 17788363131690628232: accuracy = 0.8975 (prev: 0.8975, participated)
  Client 5822289512950929004: accuracy = 0.8992 (prev: 0.8991666666666667, participated)
  Client 13263297433801258679: accuracy = 0.9217 (prev: 0.9216666666666666, participated)
Round 20 aggregated accuracy: 0.9008


# **Method 5: Top-K Client Selection**

This method selects a fixed number (K=3) of clients per round, either randomly or based on the lowest accuracies, to balance diversity and focus in training.

In [ ]:
# If no clients are selected select K clients for training
import random
from typing import Dict, List, Optional, Tuple

import flwr as fl
from flwr.common import (
    Parameters,
    Scalar,
    FitIns,
    FitRes,
    EvaluateRes,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy

class New(fl.server.strategy.FedAvg):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.client_accuracies: Dict[str, float] = {}
        self.previous_client_accuracies: Dict[str, Optional[float]] = {}
        self.communication_count = 0
        self.terminated = False

    def initialize_clients(self, client_manager: ClientManager):
        for client_proxy in client_manager.all().values():
            cid = client_proxy.cid
            # Initialize previous accuracies to None
            self.previous_client_accuracies[cid] = None
            self.client_accuracies[cid] = 0.0

    def configure_fit(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager,
    ) -> List[Tuple[ClientProxy, FitIns]]:
        if self.terminated:
            return []

        if server_round == 1:
            self.initialize_clients(client_manager)

        all_clients = list(client_manager.all().values())
        selected_clients = []

        print(f"\nRound {server_round}:")
        for client in all_clients:
            cid = client.cid

            current_accuracy = self.client_accuracies.get(cid, 0.0)
            previous_accuracy = self.previous_client_accuracies.get(cid)

            # Determine if the client is underperforming
            if previous_accuracy is None:
                # First round, consider all clients as underperforming
                underperforming = True
            else:
                if current_accuracy <= previous_accuracy:
                    underperforming = True
                else:
                    underperforming = False

            if underperforming:
                # Client is underperforming, select for training
                print(f"  Client {cid}: Underperforming (prev: {previous_accuracy}, current: {current_accuracy}), selected")
                selected_clients.append(client)
            else:
                # Client is performing well, do not select for training
                print(f"  Client {cid}: Performing well (prev: {previous_accuracy}, current: {current_accuracy}), not selected")

        if not selected_clients:
            print("No clients selected based on criteria.")

            # Option (i): Randomly select k clients
            k = 3  # Adjust k as needed
            selected_clients = random.sample(all_clients, k)
            print(f"  Randomly selected {k} clients.")

            # Option (ii): Select top-k clients with the lowest accuracies
            # Uncomment the following lines to use Option (ii)
            """
            k = 3  # Adjust k as needed
            clients_sorted = sorted(all_clients, key=lambda c: self.client_accuracies.get(c.cid, 0.0))
            selected_clients = clients_sorted[:k]
            print(f"  Selected top-{k} clients with the lowest accuracies.")
            """

        self.communication_count += len(selected_clients)
        print(f"  Number of clients selected: {len(selected_clients)}")

        # Prepare fit instructions
        config = {}
        fit_ins = FitIns(parameters, config)
        return [(client, fit_ins) for client in selected_clients]

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        # Call super to aggregate parameters as usual
        aggregated_parameters, aggregated_metrics = super().aggregate_fit(server_round, results, failures)
        if aggregated_parameters is not None:
            self.latest_parameters = aggregated_parameters
        return aggregated_parameters, aggregated_metrics

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[BaseException],
    ) -> Optional[float]:
        metrics = []
        participating_cids = [client_proxy.cid for client_proxy, _ in results]

        for client_proxy, evaluate_res in results:
            cid = client_proxy.cid
            num_examples = evaluate_res.num_examples
            accuracy = evaluate_res.metrics.get("accuracy", 0.0)

            # Update previous accuracy
            self.previous_client_accuracies[cid] = self.client_accuracies.get(cid, 0.0)
            # Update current accuracy
            self.client_accuracies[cid] = accuracy

            metrics.append((num_examples, {"accuracy": accuracy}))

        print(f"\nRound {server_round} client accuracies:")
        for cid in self.client_accuracies.keys():
            current_accuracy = self.client_accuracies[cid]
            previous_accuracy = self.previous_client_accuracies[cid]
            status = "participated" if cid in participating_cids else "did not participate"
            print(f"  Client {cid}: accuracy = {current_accuracy:.4f} (prev: {previous_accuracy}, {status})")

        aggregated_metrics = self.weighted_average(metrics)
        global_accuracy = aggregated_metrics["accuracy"]
        print(f"Round {server_round} aggregated accuracy: {global_accuracy:.4f}")

        return super().aggregate_evaluate(server_round, results, failures)

    @staticmethod
    def weighted_average(metrics: List[Tuple[int, Dict[str, Scalar]]]) -> Dict[str, Scalar]:
        accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
        examples = [num_examples for num_examples, _ in metrics]
        if sum(examples) == 0:
            return {"accuracy": 0.0}
        return {"accuracy": sum(accuracies) / sum(examples)}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
NUM_CLIENTS = 10
NUM_ROUNDS = 20

strategy5 = New(
    fraction_fit=1.0,         # All clients are considered in selection
    min_fit_clients=1,        # At least one client should be available
    min_available_clients=NUM_CLIENTS,
)


In [ ]:
server_config5 = ServerConfig(num_rounds=NUM_ROUNDS)

# Start the simulation
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    client_resources={"num_gpus": 0.5 if torch.cuda.is_available() else 0},
    config=server_config5,
    strategy=strategy5,
)

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout
2024-12-27 09:45:24,878	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'object_store_memory': 3982146355.0, 'accelerator_type:T4': 1.0, 'CPU': 2.0, 'node:172.28.0.12': 1.0, 'GPU': 1.0, 'memory': 7964292711.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
<ipython-input-31-d371e0ccd979>:4: UserWarning: No `num_cpus` specified in `client_resources`. Using `num_c


Round 1:
  Client 8336808736770875101: Underperforming (prev: None, current: 0.0), selected
  Client 15047806029616165531: Underperforming (prev: None, current: 0.0), selected
  Client 14038575247495259463: Underperforming (prev: None, current: 0.0), selected
  Client 631311497758566455: Underperforming (prev: None, current: 0.0), selected
  Client 10442842480757513332: Underperforming (prev: None, current: 0.0), selected
  Client 11183981266808082887: Underperforming (prev: None, current: 0.0), selected
  Client 5975428327333756466: Underperforming (prev: None, current: 0.0), selected
  Client 6001700192988651239: Underperforming (prev: None, current: 0.0), selected
  Client 16761844778504997753: Underperforming (prev: None, current: 0.0), selected
  Client 3323442546069927317: Underperforming (prev: None, current: 0.0), selected
  Number of clients selected: 10


(ClientAppActor pid=20602) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(ClientAppActor pid=20602) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=20602) 
(ClientAppActor pid=20602)             This is a deprecated feature. It will be removed
(ClientAppActor pid=20602)             entirely in future versions of Flower.
(ClientAppActor pid=20602)         
(ClientAppActor pid=20601) /usr/local/lib/python3.10/dist-packages/jupyter_client/connect.py:28: DeprecationWarning: Jupyter is migrating its paths to use standard platformdirs
(ClientAppActor pid=20601) given by the platformdirs library.  To remove this


Round 1 client accuracies:
  Client 8336808736770875101: accuracy = 0.9258 (prev: 0.0, participated)
  Client 15047806029616165531: accuracy = 0.9350 (prev: 0.0, participated)
  Client 14038575247495259463: accuracy = 0.9242 (prev: 0.0, participated)
  Client 631311497758566455: accuracy = 0.9258 (prev: 0.0, participated)
  Client 10442842480757513332: accuracy = 0.9267 (prev: 0.0, participated)
  Client 11183981266808082887: accuracy = 0.9242 (prev: 0.0, participated)
  Client 5975428327333756466: accuracy = 0.9200 (prev: 0.0, participated)
  Client 6001700192988651239: accuracy = 0.9192 (prev: 0.0, participated)
  Client 16761844778504997753: accuracy = 0.9175 (prev: 0.0, participated)
  Client 3323442546069927317: accuracy = 0.9408 (prev: 0.0, participated)
Round 1 aggregated accuracy: 0.9259

Round 2:
  Client 8336808736770875101: Performing well (prev: 0.0, current: 0.9258333333333333), not selected
  Client 15047806029616165531: Performing well (prev: 0.0, current: 0.935), not s

(ClientAppActor pid=20602)          [repeated 6x across cluster]
(ClientAppActor pid=20602) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 3x across cluster]
(ClientAppActor pid=20602)             This is a deprecated feature. It will be removed [repeated 3x across cluster]
(ClientAppActor pid=20602)             entirely in future versions of Flower. [repeated 3x across cluster]
(ClientAppActor pid=20602) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 3x across cluster]
INFO :      aggregate_fit: received 3 results and 0 failures
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)
(Cli

...


Round 18 client accuracies:
  Client 8336808736770875101: accuracy = 0.9883 (prev: 0.9866666666666667, participated)
  Client 15047806029616165531: accuracy = 0.9892 (prev: 0.9825, participated)
  Client 14038575247495259463: accuracy = 0.9917 (prev: 0.9916666666666667, participated)
  Client 631311497758566455: accuracy = 0.9825 (prev: 0.9833333333333333, participated)
  Client 10442842480757513332: accuracy = 0.9783 (prev: 0.9791666666666666, participated)
  Client 11183981266808082887: accuracy = 0.9925 (prev: 0.9941666666666666, participated)
  Client 5975428327333756466: accuracy = 0.9842 (prev: 0.985, participated)
  Client 6001700192988651239: accuracy = 0.9842 (prev: 0.9825, participated)
  Client 16761844778504997753: accuracy = 0.9892 (prev: 0.9908333333333333, participated)
  Client 3323442546069927317: accuracy = 0.9858 (prev: 0.9875, participated)
Round 18 aggregated accuracy: 0.9866

Round 19:
  Client 8336808736770875101: Performing well (prev: 0.9866666666666667, curre

(ClientAppActor pid=20601) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 4x across cluster]
(ClientAppActor pid=20602)          [repeated 8x across cluster]
(ClientAppActor pid=20602) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 4x across cluster]
(ClientAppActor pid=20602)             This is a deprecated feature. It will be removed [repeated 4x across cluster]
(ClientAppActor pid=20602)             entirely in future versions of Flower. [repeated 4x across cluster]
(ClientAppActor pid=20602) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance


Round 19 client accuracies:
  Client 8336808736770875101: accuracy = 0.9892 (prev: 0.9883333333333333, participated)
  Client 15047806029616165531: accuracy = 0.9900 (prev: 0.9891666666666666, participated)
  Client 14038575247495259463: accuracy = 0.9900 (prev: 0.9916666666666667, participated)
  Client 631311497758566455: accuracy = 0.9825 (prev: 0.9825, participated)
  Client 10442842480757513332: accuracy = 0.9792 (prev: 0.9783333333333334, participated)
  Client 11183981266808082887: accuracy = 0.9933 (prev: 0.9925, participated)
  Client 5975428327333756466: accuracy = 0.9858 (prev: 0.9841666666666666, participated)
  Client 6001700192988651239: accuracy = 0.9850 (prev: 0.9841666666666666, participated)
  Client 16761844778504997753: accuracy = 0.9867 (prev: 0.9891666666666666, participated)
  Client 3323442546069927317: accuracy = 0.9858 (prev: 0.9858333333333333, participated)
Round 19 aggregated accuracy: 0.9868

Round 20:
  Client 8336808736770875101: Performing well (prev: 

(ClientAppActor pid=20602) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`. [repeated 3x across cluster]
(ClientAppActor pid=20601)          [repeated 8x across cluster]
(ClientAppActor pid=20601) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context` [repeated 4x across cluster]
(ClientAppActor pid=20601)             This is a deprecated feature. It will be removed [repeated 4x across cluster]
(ClientAppActor pid=20601)             entirely in future versions of Flower. [repeated 4x across cluster]
INFO :      aggregate_fit: received 4 results and 0 failures
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)
(Cli


Round 20 client accuracies:
  Client 8336808736770875101: accuracy = 0.9850 (prev: 0.9891666666666666, participated)
  Client 15047806029616165531: accuracy = 0.9875 (prev: 0.99, participated)
  Client 14038575247495259463: accuracy = 0.9942 (prev: 0.99, participated)
  Client 631311497758566455: accuracy = 0.9842 (prev: 0.9825, participated)
  Client 10442842480757513332: accuracy = 0.9767 (prev: 0.9791666666666666, participated)
  Client 11183981266808082887: accuracy = 0.9917 (prev: 0.9933333333333333, participated)
  Client 5975428327333756466: accuracy = 0.9850 (prev: 0.9858333333333333, participated)
  Client 6001700192988651239: accuracy = 0.9833 (prev: 0.985, participated)
  Client 16761844778504997753: accuracy = 0.9875 (prev: 0.9866666666666667, participated)
  Client 3323442546069927317: accuracy = 0.9883 (prev: 0.9858333333333333, participated)
Round 20 aggregated accuracy: 0.9863


# **Method 6: Dynamic K and Alpha Selection**

This method dynamically adjusts client participation using K and alpha parameters to balance exploration and exploitation, optimizing accuracy and communication efficiency.

In [ ]:
# Install required packages
!pip install -q flwr[simulation] flwr-datasets[vision] torch torchvision matplotlib

In [ ]:
# Import statements
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from collections import OrderedDict
from typing import List, Tuple, Dict, Optional

from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader

import flwr as fl
from flwr.client import Client, NumPyClient
from flwr.common import (
    Metrics,
    Parameters,
    Scalar,
    FitIns,
    FitRes,
    EvaluateRes,
    Config,
)
from flwr.server import ServerConfig
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy import FedAvg
from flwr.common.logger import log
from logging import INFO
from flwr_datasets import FederatedDataset


In [ ]:
# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Device configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")


disable_progress_bar()

Training on cpu


In [ ]:
# Global constants
NUM_CLIENTS = 10
NUM_ROUNDS = 20
BATCH_SIZE = 32
INITIAL_ROUNDS = 5  # Minimum number of rounds to enforce initial selection
K = 3  # Number of clients to select per round
ALPHA_VALUES = [0.5, 0.2, 0.1]  # Different alpha values for experimentation

In [ ]:
# Function to load MNIST dataset
def load_datasets(partition_id: int):
    fds = FederatedDataset(dataset="mnist", partitioners={"train": NUM_CLIENTS})
    partition = fds.load_partition(partition_id)
    # Divide data on each node: 80% train, 20% test
    partition_train_test = partition.train_test_split(test_size=0.2, seed=42)
    pytorch_transforms = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )

    def apply_transforms(batch):
        batch["image"] = [pytorch_transforms(img) for img in batch["image"]]
        return batch

    # Create train/val for each partition and wrap it into DataLoader
    partition_train_test = partition_train_test.with_transform(apply_transforms)
    trainloader = DataLoader(
        partition_train_test["train"], batch_size=BATCH_SIZE, shuffle=True
    )
    valloader = DataLoader(partition_train_test["test"], batch_size=BATCH_SIZE)
    testset = fds.load_split("test").with_transform(apply_transforms)
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloader, valloader, testloader

In [ ]:
# CNN Model For MNIST
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)

        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
# Training function
def train(net, trainloader, epochs: int):

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for _ in range(epochs):
        for batch in trainloader:
            images, labels = batch["image"].to(DEVICE), batch["label"].to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# Evaluation function
def test(net, testloader):

    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for batch in testloader:
            images, labels = batch["image"].to(DEVICE), batch["label"].to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader)
    accuracy = correct / total
    return loss, accuracy

In [ ]:
# Functions to set and get model parameters
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

In [ ]:
# Flower client implementation
class FlowerClient(NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config: Config) -> List[np.ndarray]:
        return get_parameters(self.net)

    def fit(
        self, parameters: List[np.ndarray], config: Config
    ) -> Tuple[List[np.ndarray], int, Dict]:
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader.dataset), {}

    def evaluate(
        self, parameters: List[np.ndarray], config: Config
    ) -> Tuple[float, int, Dict]:
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader.dataset), {"accuracy": float(accuracy)}

    def to_client(self) -> Client:
        """Convert NumPyClient to Client."""
        return super().to_client()

In [ ]:
from flwr.common import Context
# Client function
def client_fn(cid: str) -> Client:

    # Load model
    net = Net().to(DEVICE)
    # Load data (each client has its own partition)
    partition_id = int(cid)
    trainloader, valloader, _ = load_datasets(partition_id)
    # Create and return the Flower client
    return FlowerClient(cid, net, trainloader, valloader).to_client()

Strategy for Method 6

In [ ]:
from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from typing import Dict, List, Optional, Tuple
import numpy as np
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy import Strategy
from flwr.common.logger import log
from logging import INFO, WARNING, ERROR

class CS_final(Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
        evaluate_fn=None,
        on_fit_config_fn=None,
        on_evaluate_config_fn=None,
        accept_failures: bool = True,
        initial_rounds: int = 5,  # Minimum number of initial rounds
        k: int = 3,               # Number of clients to select per round
        alpha: float = 0.5,       # Smoothing factor for selection probability
        total_rounds: int = 20,   # Total number of rounds
    ):
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.evaluate_fn = evaluate_fn
        self.on_fit_config_fn = on_fit_config_fn
        self.on_evaluate_config_fn = on_evaluate_config_fn
        self.accept_failures = accept_failures

        self.initial_rounds = initial_rounds
        self.k = k
        self.alpha = alpha
        self.client_accuracies: Dict[str, float] = {}
        self.current_round = 0
        self.parameters: Optional[Parameters] = None  # Will be set after initialization
        self.global_accuracy: float = 0.0  # Store global model accuracy

        self.total_rounds = total_rounds
        self.total_communications = 0     # Initialize total communications

        # Track clients selected in initial rounds to ensure all participate at least once
        self.initial_selected_clients: set = set()

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        # Return None to let the server initialize parameters from a client
        return None

    def configure_fit(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager,
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""
        self.current_round = server_round
        log(INFO, f"\n[ROUND {server_round}]")

        clients_dict = client_manager.all()
        available_cids = list(clients_dict.keys())

        # Ensure all clients have an accuracy value
        for cid in available_cids:
            if cid not in self.client_accuracies:
                self.client_accuracies[cid] = 0.0  # Default accuracy

        selected_cids = []

        if server_round <= self.initial_rounds:
            # Select top-k least accurate clients that haven't been selected yet
            # Sort clients by least accuracy
            sorted_clients = sorted(
                self.client_accuracies.items(), key=lambda x: x[1]
            )
            # Select clients not yet selected
            available_for_selection = [
                cid for cid, _ in sorted_clients if cid not in self.initial_selected_clients
            ]
            # Select up to k clients
            for cid, _ in sorted_clients:
                if cid not in self.initial_selected_clients:
                    selected_cids.append(cid)
                    self.initial_selected_clients.add(cid)
                    if len(selected_cids) == self.k:
                        break
            # If not enough unique clients, fill the rest with top-k least accurate
            if len(selected_cids) < self.k:
                additional = self.k - len(selected_cids)
                for cid, _ in sorted_clients:
                    if cid not in selected_cids:
                        selected_cids.append(cid)
                        if len(selected_cids) == self.k:
                            break
            log(
                INFO,
                f"Round {server_round}: Initial selection - Selected clients {selected_cids}",
            )
        else:
            # Probabilistic selection based on 1 - alpha * accuracy
            probabilities = {}
            for cid in available_cids:
                accuracy = self.client_accuracies[cid]
                prob = max(1 - self.alpha * accuracy, 0.0)
                probabilities[cid] = prob

            # Normalize probabilities
            total_prob = sum(probabilities.values())
            if total_prob == 0:
                # If total_prob is zero, default to uniform probabilities
                for cid in probabilities:
                    probabilities[cid] = 1.0 / len(probabilities)
            else:
                for cid in probabilities:
                    probabilities[cid] /= total_prob

            # Select k unique clients based on probabilities
            selected_cids = list(
                np.random.choice(
                    available_cids,
                    size=self.k,
                    replace=False,
                    p=[probabilities[cid] for cid in available_cids],
                )
            )
            log(
                INFO,
                f"Round {server_round}: Probabilistic selection - Selected clients {selected_cids}",
            )

        # Update total communications
        self.total_communications += len(selected_cids)

        clients = [
            clients_dict.get(cid) for cid in selected_cids if clients_dict.get(cid) is not None
        ]

        # Create fit instructions
        config = self.on_fit_config_fn(server_round) if self.on_fit_config_fn else {}
        fit_ins = FitIns(parameters, config)

        # Return client configurations as a list of (ClientProxy, FitIns)
        return [(client, fit_ins) for client in clients if client is not None]

    def configure_evaluate(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager,
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        # Evaluate on all clients
        clients = list(client_manager.all().values())

        # Create evaluate instructions
        config = (
            self.on_evaluate_config_fn(server_round)
            if self.on_evaluate_config_fn
            else {}
        )
        evaluate_ins = EvaluateIns(parameters, config)

        # Return client configurations as a list of (ClientProxy, EvaluateIns)
        return [(client, evaluate_ins) for client in clients if client is not None]

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""
        if not results:
            return None, {}

        # Total number of examples used for training
        total_examples = sum([res.num_examples for _, res in results])

        # Initialize the list to store weighted updates
        weighted_updates = []

        for _, fit_res in results:
            # Deserialize parameters to ndarrays
            client_weights = parameters_to_ndarrays(fit_res.parameters)
            # Calculate the weight based on the number of examples
            num_examples = fit_res.num_examples
            weight = num_examples / total_examples
            # Append the weighted client weights
            weighted_updates.append([layer * weight for layer in client_weights])

        # Sum the weighted updates
        aggregated_weights = [
            np.sum([update[layer] for update in weighted_updates], axis=0)
            for layer in range(len(weighted_updates[0]))
        ]

        # Serialize aggregated weights back to Parameters
        aggregated_parameters = ndarrays_to_parameters(aggregated_weights)

        # Return aggregated parameters and an empty metrics dictionary
        return aggregated_parameters, {}

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation results."""
        if not results:
            return None, {}

        # Compute weighted loss
        losses = [res.num_examples * res.loss for _, res in results]
        examples = [res.num_examples for _, res in results]
        aggregated_loss = sum(losses) / sum(examples)

        # Compute weighted accuracy
        accuracies = [
            res.num_examples * res.metrics.get("accuracy", 0.0)
            for _, res in results
        ]
        aggregated_accuracy = sum(accuracies) / sum(examples)
        self.global_accuracy = aggregated_accuracy  # Store global accuracy

        # Update client accuracies
        for client_proxy, eval_res in results:
            cid = client_proxy.cid
            accuracy = eval_res.metrics.get("accuracy", 0.0)
            prev_accuracy = self.client_accuracies.get(cid, 0.0)
            self.client_accuracies[cid] = accuracy

            # Log client accuracies
            log(
                INFO,
                f"Client {cid}: accuracy = {accuracy:.4f} (prev: {prev_accuracy:.4f})",
            )

        # Log aggregated accuracy
        log(
            INFO,
            f"Round {server_round} aggregated accuracy: {aggregated_accuracy:.4f}\n",
        )

        # Print total communications after the last round
        if server_round == self.total_rounds:
            log(
                INFO,
                f"[SUMMARY]\nTotal communication rounds: {self.total_communications}",
            )

        # Return aggregated loss and metrics
        return aggregated_loss, {"accuracy": aggregated_accuracy}

    def evaluate(
        self,
        server_round: int,
        parameters: Parameters,
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate model parameters using an optional validation function."""
        if self.evaluate_fn is None:
            # No validation function provided
            return None
        return self.evaluate_fn(server_round, parameters)


In [ ]:
# Start simulation
if __name__ == "__main__":
    # Iterate over different alpha values for experimentation
    for alpha in ALPHA_VALUES:
        print(f"\nStarting simulation with alpha = {alpha}\n{'='*40}")

        # Create strategy instance
        strategy_strategy_final = CS_final(
            fraction_fit=1.0,
            fraction_evaluate=1.0,
            min_fit_clients=2,
            min_evaluate_clients=2,
            min_available_clients=2,
            initial_rounds=INITIAL_ROUNDS,
            k=K,
            alpha=alpha,
            total_rounds=20,
        )

        # Start the simulation
        fl.simulation.start_simulation(
            client_fn=client_fn,
            num_clients=NUM_CLIENTS,
            config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
            strategy=strategy_strategy_final,
        )

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout



Starting simulation with alpha = 0.5


2024-12-27 11:07:06,805	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 8001213236.0, 'object_store_memory': 4000606617.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=34711) 2024-12-27 11:07:11.275005: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=34711) 2024-12-27 11:07:11.314342: E external/local_xla/xla/s


Starting simulation with alpha = 0.2


2024-12-27 11:16:53,913	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'memory': 8002284750.0, 'object_store_memory': 4001142374.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=39629) 2024-12-27 11:16:58.581197: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=39629) 2024-12-27 11:16:58.620965: E external/local_xla/xla/s


Starting simulation with alpha = 0.1


2024-12-27 11:27:10,828	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 4000992460.0, 'memory': 8001984923.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=44663) 2024-12-27 11:27:17.658853: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=44663) 2024-12-27 11:27:17.698436: E external/local_xla/xla/s

In [ ]:
# Start simulation
if __name__ == "__main__":
    # Set desired k and alpha values
    K = 3
    ALPHA = 0.5  # Change Alpha value as needed

    print(f"\nStarting simulation with k = {K} and alpha = {ALPHA}\n{'='*50}")

    # Create strategy instance with desired parameters
    strategy_strategy_final = CS_final(
        fraction_fit=1.0,
        fraction_evaluate=1.0,
        min_fit_clients=2,
        min_evaluate_clients=2,
        min_available_clients=2,
        initial_rounds=INITIAL_ROUNDS,
        k=K,
        alpha=ALPHA,
        total_rounds=20,
    )

    # Start the simulation
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=NUM_CLIENTS,
        config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
        strategy=strategy_strategy_final,
    )


# **Visualisation**
For Comparing Accuracy over Communication rounds

In [ ]:
import itertools
import csv
import flwr as fl
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy import Strategy
from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from typing import Dict, List, Optional, Tuple
import numpy as np
from flwr.common.logger import log
from logging import INFO, WARNING, ERROR
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Define  necessary variables
NUM_CLIENTS = 10
INITIAL_ROUNDS = 5
NUM_ROUNDS = 20

class CS_final(Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
        evaluate_fn=None,
        on_fit_config_fn=None,
        on_evaluate_config_fn=None,
        accept_failures: bool = True,
        initial_rounds: int = 5,  # Minimum number of initial rounds
        k: int = 3,               # Number of clients to select per round
        alpha: float = 0.5,       # Smoothing factor for selection probability
        total_rounds: int = 20,   # Total number of rounds
    ):
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.evaluate_fn = evaluate_fn
        self.on_fit_config_fn = on_fit_config_fn
        self.on_evaluate_config_fn = on_evaluate_config_fn
        self.accept_failures = accept_failures

        self.initial_rounds = initial_rounds
        self.k = k
        self.alpha = alpha
        self.client_accuracies: Dict[str, float] = {}
        self.current_round = 0
        self.parameters: Optional[Parameters] = None  # Will be set after initialization
        self.global_accuracy: float = 0.0  # Store global model accuracy

        self.total_rounds = total_rounds
        self.total_communications = 0     # Initialize total communications

        # Track clients selected in initial rounds to ensure all participate at least once
        self.initial_selected_clients: set = set()

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        # Return None to let the server initialize parameters from a client
        return None

    def configure_fit(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager,
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""
        self.current_round = server_round
        log(INFO, f"\n[ROUND {server_round}]")

        clients_dict = client_manager.all()
        available_cids = list(clients_dict.keys())

        # Ensure all clients have an accuracy value
        for cid in available_cids:
            if cid not in self.client_accuracies:
                self.client_accuracies[cid] = 0.0

        selected_cids = []

        if server_round <= self.initial_rounds:
            # Select top-k least accurate clients that haven't been selected yet
            # Sort clients by least accuracy
            sorted_clients = sorted(
                self.client_accuracies.items(), key=lambda x: x[1]
            )
            # Select clients not yet selected
            available_for_selection = [
                cid for cid, _ in sorted_clients if cid not in self.initial_selected_clients
            ]
            # Select up to k clients
            for cid, _ in sorted_clients:
                if cid not in self.initial_selected_clients:
                    selected_cids.append(cid)
                    self.initial_selected_clients.add(cid)
                    if len(selected_cids) == self.k:
                        break
            # If not enough unique clients, fill the rest with top-k least accurate
            if len(selected_cids) < self.k:
                additional = self.k - len(selected_cids)
                for cid, _ in sorted_clients:
                    if cid not in selected_cids:
                        selected_cids.append(cid)
                        if len(selected_cids) == self.k:
                            break
            log(
                INFO,
                f"Round {server_round}: Initial selection - Selected clients {selected_cids}",
            )
        else:
            # Probabilistic selection based on 1 - alpha * accuracy
            probabilities = {}
            for cid in available_cids:
                accuracy = self.client_accuracies[cid]
                prob = max(1 - self.alpha * accuracy, 0.0)
                probabilities[cid] = prob

            # Normalize probabilities
            total_prob = sum(probabilities.values())
            if total_prob == 0:
                # If total_prob is zero, default to uniform probabilities
                for cid in probabilities:
                    probabilities[cid] = 1.0 / len(probabilities)
            else:
                for cid in probabilities:
                    probabilities[cid] /= total_prob

            # Select k unique clients based on probabilities
            selected_cids = list(
                np.random.choice(
                    available_cids,
                    size=self.k,
                    replace=False,
                    p=[probabilities[cid] for cid in available_cids],
                )
            )
            log(
                INFO,
                f"Round {server_round}: Probabilistic selection - Selected clients {selected_cids}",
            )

        # Update total communications
        self.total_communications += len(selected_cids)

        clients = [
            clients_dict.get(cid) for cid in selected_cids if clients_dict.get(cid) is not None
        ]

        # Create fit instructions
        config = self.on_fit_config_fn(server_round) if self.on_fit_config_fn else {}
        fit_ins = FitIns(parameters, config)

        # Return client configurations as a list
        return [(client, fit_ins) for client in clients if client is not None]

    def configure_evaluate(
        self,
        server_round: int,
        parameters: Parameters,
        client_manager: ClientManager,
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        # Evaluate on all clients
        clients = list(client_manager.all().values())

        # Create evaluate instructions
        config = (
            self.on_evaluate_config_fn(server_round)
            if self.on_evaluate_config_fn
            else {}
        )
        evaluate_ins = EvaluateIns(parameters, config)

        # Return client configurations as a list of (ClientProxy, EvaluateIns)
        return [(client, evaluate_ins) for client in clients if client is not None]

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""
        if not results:
            return None, {}

        # Total number of examples used for training
        total_examples = sum([res.num_examples for _, res in results])

        # Initialize the list to store weighted updates
        weighted_updates = []

        for _, fit_res in results:
            # Deserialize parameters to ndarrays
            client_weights = parameters_to_ndarrays(fit_res.parameters)
            # Calculate the weight based on the number of examples
            num_examples = fit_res.num_examples
            weight = num_examples / total_examples
            # Append the weighted client weights
            weighted_updates.append([layer * weight for layer in client_weights])

        # Sum the weighted updates
        aggregated_weights = [
            np.sum([update[layer] for update in weighted_updates], axis=0)
            for layer in range(len(weighted_updates[0]))
        ]

        # Serialize aggregated weights back to Parameters
        aggregated_parameters = ndarrays_to_parameters(aggregated_weights)

        # Return aggregated parameters and an empty metrics dictionary
        return aggregated_parameters, {}

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation results."""
        if not results:
            return None, {}

        # Compute weighted loss
        losses = [res.num_examples * res.loss for _, res in results]
        examples = [res.num_examples for _, res in results]
        aggregated_loss = sum(losses) / sum(examples)

        # Compute weighted accuracy
        accuracies = [
            res.num_examples * res.metrics.get("accuracy", 0.0)
            for _, res in results
        ]
        aggregated_accuracy = sum(accuracies) / sum(examples)
        self.global_accuracy = aggregated_accuracy  # Store global accuracy

        # Update client accuracies
        for client_proxy, eval_res in results:
            cid = client_proxy.cid
            accuracy = eval_res.metrics.get("accuracy", 0.0)
            prev_accuracy = self.client_accuracies.get(cid, 0.0)
            self.client_accuracies[cid] = accuracy

            # Log client accuracies
            log(
                INFO,
                f"Client {cid}: accuracy = {accuracy:.4f} (prev: {prev_accuracy:.4f})",
            )

        # Log aggregated accuracy
        log(
            INFO,
            f"Round {server_round} aggregated accuracy: {aggregated_accuracy:.4f}\n",
        )

        # Print total communications after the last round
        if server_round == self.total_rounds:
            log(
                INFO,
                f"[SUMMARY]\nTotal communication rounds: {self.total_communications}",
            )

        # Return aggregated loss and metrics
        return aggregated_loss, {"accuracy": aggregated_accuracy}

    def evaluate(
        self,
        server_round: int,
        parameters: Parameters,
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate model parameters using an optional validation function."""
        if self.evaluate_fn is None:
            # No validation function provided
            return None
        return self.evaluate_fn(server_round, parameters)

In [ ]:
def run_simulation(k: int, alpha: float, initial_rounds: int, total_rounds: int) -> Dict:
    """
    Runs a single federated learning simulation with given k and alpha.

    Args:
        k (int): Number of clients to select per round.
        alpha (float): Smoothing factor for selection probability.
        initial_rounds (int): Number of initial rounds to enforce top-k selection.
        total_rounds (int): Total number of communication rounds.

    Returns:
        Dict: A dictionary containing k, alpha, final_accuracy, and total_communications.
    """
    print(f"\nRunning simulation with k={k}, alpha={alpha}")

    # Initialize the strategy with current k and alpha
    strategy = CS_final(
        fraction_fit=1.0,
        fraction_evaluate=1.0,
        min_fit_clients=2,
        min_evaluate_clients=2,
        min_available_clients=2,
        initial_rounds=initial_rounds,
        k=k,
        alpha=alpha,
        total_rounds=total_rounds,
    )

    # Start the Flower simulation
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=NUM_CLIENTS,
        config=fl.server.ServerConfig(num_rounds=total_rounds),
        strategy=strategy,
    )

    # retrieve the metrics
    final_accuracy = strategy.global_accuracy
    total_communications = strategy.total_communications

    print(f"Completed simulation with k={k}, alpha={alpha}: Accuracy={final_accuracy}, Communications={total_communications}")

    return {
        'k': k,
        'alpha': alpha,
        'final_accuracy': final_accuracy,
        'total_communications': total_communications
    }

Plot the graph

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

def main():
    # Define the range of k and alpha values

    k_values = [2, 3, 5, 7]
    alpha_values = [0.1, 0.2, 0.5, 0.8]

    # Initialize a list to store experiment results
    experiment_results = []

    # Iterate over all combinations of k and alpha
    for k, alpha in itertools.product(k_values, alpha_values):
        result = run_simulation(
            k=k,
            alpha=alpha,
            initial_rounds=INITIAL_ROUNDS,
            total_rounds=NUM_ROUNDS
        )
        experiment_results.append(result)

    # Save the results to a .csv file
    csv_file = 'experiment_results.csv'
    df = pd.DataFrame(experiment_results)
    df.to_csv(csv_file, index=False)
    print(f"\nAll experiments completed. Results saved to {csv_file}")

    # Proceed to visualization
    visualize_results(df)

def visualize_results(df: pd.DataFrame):


    # Create a  label for legend entries
    df['params'] = df.apply(lambda row: f"k={row['k']}, alpha={row['alpha']}", axis=1)

    # Create a scatter plot
    fig = px.scatter(
        df,
        x='final_accuracy',
        y='total_communications',
        color='params',
        hover_data={
            'k': True,
            'alpha': True,
            'final_accuracy': ':.4f',
            'total_communications': True
        },
        labels={
            'final_accuracy': 'Final Accuracy',
            'total_communications': 'Total Communication Rounds',
            'params': 'Parameters'
        },
        title='Federated Learning Performance: Accuracy vs Communication Rounds'
    )


    fig.update_traces(
        marker=dict(
            size=12,
            opacity=0.8,
            line=dict(width=1, color='DarkSlateGrey')
        )
    )


    fig.update_layout(
        legend_title_text='Parameters',
        xaxis_title='Final Accuracy',
        yaxis_title='Total Communication Rounds',
        hovermode='closest',
        margin=dict(l=60, r=200, t=60, b=60),
        legend=dict(
            x=1.05,
            y=1,
            xanchor='left',
            yanchor='top',
            bgcolor='rgba(255,255,255,0.9)',
            bordercolor='rgba(0,0,0,0.1)',
            borderwidth=1
        )
    )

    # Save to .HTML and show
    fig.write_html('interactive_plot.html')
    fig.show()


if __name__ == "__main__":
    main()

	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=20, no round_timeout



Running simulation with k=2, alpha=0.1


2024-12-27 12:16:54,842	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 4001585971.0, 'memory': 8003171943.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=61513) 2024-12-27 12:17:02.078865: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=61514) 2024-12-27 12:17:02.156393: E external/local_xla/xla/s

Completed simulation with k=2, alpha=0.1: Accuracy=0.9833333333333333, Communications=40

Running simulation with k=2, alpha=0.2


2024-12-27 12:25:06,835	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'object_store_memory': 4001083392.0, 'memory': 8002166784.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=65865) 2024-12-27 12:25:11.183892: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=65865) 2024-12-27 12:25:11.235068: E external/local_xla/xla/s

...

Completed simulation with k=7, alpha=0.2: Accuracy=0.9864166666666667, Communications=140

Running simulation with k=7, alpha=0.5


2024-12-27 14:51:13,199	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0, 'object_store_memory': 3995358412.0, 'memory': 7990716827.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=136679) 2024-12-27 14:51:17.844767: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=136679) 2024-12-27 14:51:17.937501: E external/local_xla/xla

Completed simulation with k=7, alpha=0.5: Accuracy=0.9869166666666667, Communications=140

Running simulation with k=7, alpha=0.8


2024-12-27 15:07:01,074	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0, 'object_store_memory': 3994605158.0, 'memory': 7989210318.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=143756) 2024-12-27 15:07:05.707060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=143756) 2024-12-27 15:07:05.748999: E external/local_xla/xla

Completed simulation with k=7, alpha=0.8: Accuracy=0.98675, Communications=140

All experiments completed. Results saved to experiment_results.csv
